In [1]:
# Enable all CPU cores
from tqdm import tqdm
import pandas as pd
import re
import pandas as pd

def parse_to_datafame(file_path):
  with open(file_path, "r") as file:
      content = file.read()
  # Define a regex pattern to extract the test case and its details
  pattern = r"\s+nat\s+(?P<nat>\d+)\s+nprim\s+(?P<nprim>\d+)\s+maxl\s+(?P<maxl>\d+)\s+max_nnl\s+(?P<max_nnl>\d+)\s+mean_nnl\s+(?P<mean_nnl>\d+)\s+max_nsh\s+(?P<max_nsh>\d+)\s+cpu_time\s+(?P<cpu_time>[\d.]+)\s+gpu_gb_in\s+(?P<gpu_gb_in>[\d.]+)\s+gpu_gb_out\s+(?P<gpu_gb_out>[\d.]+)\s+gpu_gb_total\s+(?P<gpu_gb_total>[\d.]+)\s+gpu_between_atoms\s+(?P<gpu_between_atoms>[\d.]+)\s+gpu_in_atoms\s+(?P<gpu_in_atoms>[\d.]+)\s+gpu_time\s+(?P<gpu_time>[\d.]+)\s+gpu_walltime\s+(?P<gpu_walltime>[\d.]+)\s+batch_size\s+(?P<batch_size>[\d.]+)"


  # Use re.finditer to extract all matches
  matches = re.finditer(pattern, content)
  # Create a list of dictionaries to store the extracted data
  data = []
  for match in matches:
      data.append(match.groupdict())

  # Convert the list of dictionaries into a pandas DataFrame
  df = pd.DataFrame(data)

# Convert numeric columns to appropriate data types
  numeric_columns = [
      'nat', 'nprim', 'maxl', 'max_nnl', 'mean_nnl', 'max_nsh',
      'cpu_time', 'gpu_gb_in', 'gpu_gb_out', 'gpu_gb_total',
      'gpu_between_atoms', 'gpu_in_atoms', 'gpu_time', 'gpu_walltime', "batch_size"
  ]
  # Convert the numeric columns to float
  df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

  # Display the DataFrame
  df = df.sort_values('nat')
  return df

option = 'h100_batch'
!mkdir -p notes/capture/{option}
!mkdir -p notes/data/{option}
ncore = str(1)
!OMP_NUM_THREADS={ncore} meson test -C build/ hamiltonian-batch --verbose -t 0 > notes/capture/{option}/{ncore}cpu.txt
df = parse_to_datafame(f"notes/capture/{option}/{ncore}cpu.txt")
df.to_csv(f"notes/data/{option}/{ncore}cpu.csv", index=False)
df

,nat,nprim,maxl,max_nnl,mean_nnl,max_nsh,cpu_time,gpu_gb_in,gpu_gb_out,gpu_gb_total,gpu_between_atoms,gpu_in_atoms,gpu_time,gpu_walltime,batch_size
0,43,6,1,42,21,2,16.859063,0.00106,0.001046,0.002105,12.894208,1.418016,14.312223,1527.557689,1
1,43,6,1,42,21,2,10.423970,0.00106,0.001046,0.002105,6.545408,1.294336,14.385152,16.345268,2
2,43,6,1,42,21,2,10.638661,0.00106,0.001046,0.002105,6.148096,1.235872,25.828257,27.570745,4
3,43,6,1,42,21,2,10.337539,0.00106,0.001046,0.002105,5.977312,1.292288,49.110786,50.945091,8
4,43,6,1,42,21,2,10.470365,0.00106,0.001046,0.002105,5.928896,1.315808,96.178146,98.264438,16
5,43,6,1,42,21,2,10.289750,0.00106,0.001046,0.002105,5.240640,1.058944,168.759430,170.577072,32
6,43,6,1,42,21,2,10.438380,0.00106,0.001046,0.002105,4.270209,1.000448,274.293854,276.150709,64
7,43,6,1,42,21,2,10.461734,0.00106,0.001046,0.002105,4.289248,0.990176,550.013977,552.111162,128
8,43,6,1,42,21,2,10.552004,0.00106,0.001046,0.002105,4.296768,1.128448,1101.100952,1102.836003,256
9,43,6,1,42,21,2,10.622014,0.00106,0.001046,0.002105,4.299345,1.089536,2202.354248,2204.307258,512


In [4]:
!nproc
!lscpu
!nvidia-smi

208


Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 57 bits virtual
  Byte Order:             Little Endian
CPU(s):                   208
  On-line CPU(s) list:    0-207
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) Platinum 8470
    CPU family:           6
    Model:                143
    Thread(s) per core:   2
    Core(s) per socket:   52
    Socket(s):            2
    Stepping:             8
    Frequency boost:      enabled
    CPU max MHz:          2001.0000
    CPU min MHz:          800.0000
    BogoMIPS:             4000.00
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 s
                          s ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc 
                          art arch_perfmon pebs bts rep_good nopl xtopology nons
                          top_tsc 